# Agent Memory: Building Memory-Enabled Investment Agents with LangGraph

In this notebook, we'll explore **agent memory systems** - the ability for AI agents to remember information across interactions. We'll implement all five memory types from the **CoALA (Cognitive Architectures for Language Agents)** framework while building a Stone Ridge Investment Advisory Assistant.

**Learning Objectives:**
- Understand the 5 memory types from the CoALA framework
- Implement short-term memory with checkpointers and thread_id
- Build long-term memory with InMemoryStore and namespaces
- Use semantic memory for meaning-based retrieval
- Apply episodic memory for few-shot learning from past experiences
- Create procedural memory for self-improving agents
- Combine all memory types into a unified investment advisory agent

## Table of Contents:

- **Breakout Room #1:** Memory Foundations
  - Task 1: Dependencies
  - Task 2: Understanding Agent Memory (CoALA Framework)
  - Task 3: Short-Term Memory (MemorySaver, thread_id)
  - Task 4: Long-Term Memory (InMemoryStore, namespaces)
  - Task 5: Message Trimming & Context Management
  - Question #1 & Question #2
  - 🏗️ Activity #1: Store & Retrieve User Investment Profile

- **Breakout Room #2:** Advanced Memory & Integration
  - Task 6: Semantic Memory (Embeddings + Search)
  - Task 7: Building Semantic Investment Knowledge Base
  - Task 8: Episodic Memory (Few-Shot Learning)
  - Task 9: Procedural Memory (Self-Improving Agent)
  - Task 10: Unified Investment Memory Agent
  - Question #3 & Question #4
  - 🏗️ Activity #2: Investment Memory Dashboard

---
# 🤝 Breakout Room #1
## Memory Foundations

## Task 1: Dependencies

Before we begin, make sure you have:

1. **API Keys** for:
   - OpenAI (for GPT-4o-mini and embeddings)
   - LangSmith (optional, for tracing)

2. **Dependencies installed** via `uv sync`

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

In [2]:
# Set API Keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [ ]:
# Optional: LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Agent Memory - Investment - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key (press Enter to skip): ") or ""

if not os.environ["LANGCHAIN_API_KEY"]:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")
else:
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")

In [4]:
# Initialize LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Test the connection
response = llm.invoke("Say 'Memory systems ready!' in exactly those words.")
print(response.content)

Memory systems ready!


## Task 2: Understanding Agent Memory (CoALA Framework)

The **CoALA (Cognitive Architectures for Language Agents)** framework identifies 5 types of memory that agents can use:

| Memory Type | Human Analogy | AI Implementation | Investment Example |
|-------------|---------------|-------------------|------------------|
| **Short-term** | What someone just said | Conversation history within a thread | Current investment consultation conversation |
| **Long-term** | Remembering a friend's birthday | User preferences stored across sessions | User's risk tolerance, portfolio size, investment goals |
| **Semantic** | Knowing Paris is in France | Facts retrieved by meaning | Investment knowledge retrieval |
| **Episodic** | Remembering your first day at work | Learning from past experiences | Past successful advisory patterns |
| **Procedural** | Knowing how to ride a bike | Self-improving instructions | Learned communication and advisory preferences |

### Memory Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                LangGraph Investment Advisory Agent               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐           │
│  │  Short-term  │  │  Long-term   │  │   Semantic   │           │
│  │    Memory    │  │    Memory    │  │    Memory    │           │
│  │              │  │              │  │              │           │
│  │ Checkpointer │  │    Store     │  │Store+Embed   │           │
│  │ + thread_id  │  │ + namespace  │  │  + search()  │           │
│  └──────────────┘  └──────────────┘  └──────────────┘           │
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐                             │
│  │   Episodic   │  │  Procedural  │                             │
│  │    Memory    │  │    Memory    │                             │
│  │              │  │              │                             │
│  │  Few-shot    │  │Self-modifying│                             │
│  │  examples    │  │   prompts    │                             │
│  └──────────────┘  └──────────────┘                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key LangGraph Components

| Component | Memory Type | Scope |
|-----------|-------------|-------|
| `MemorySaver` (Checkpointer) | Short-term | Within a single thread |
| `InMemoryStore` | Long-term, Semantic, Episodic, Procedural | Across all threads |
| `thread_id` | Short-term | Identifies unique conversations |
| Namespaces | All store-based | Organizes memories by user/purpose |

**Documentation:**
- [CoALA Paper](https://arxiv.org/abs/2309.02427)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)

## Task 3: Short-Term Memory (MemorySaver, thread_id)

**Short-term memory** maintains context within a single conversation thread. Think of it like your working memory during a phone call - you remember what was said earlier, but once the call ends, those details fade.

In LangGraph, short-term memory is implemented through:
- **Checkpointer**: Saves the graph state at each step
- **thread_id**: Uniquely identifies each conversation

### How It Works

```
Thread 1: "Hi, I'm Alice"          Thread 2: "What's my name?"
     │                                   │
     ▼                                   ▼
┌──────────────┐                   ┌──────────────┐
│ Checkpointer │                   │ Checkpointer │
│  thread_1    │                   │  thread_2    │
│              │                   │              │
│ ["Hi Alice"] │                   │ [empty]      │
└──────────────┘                   └──────────────┘
     │                                   │
     ▼                                   ▼
"Hi Alice!"                        "I don't know your name"
```

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state schema for our graph
# The `add_messages` annotation tells LangGraph how to update the messages list
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Define our investment chatbot node
def investment_chatbot(state: State):
    """Process the conversation and generate an investment-focused response."""
    system_prompt = SystemMessage(content="""You are a friendly Investment Advisory Assistant. 
Help users with questions about Stone Ridge's investment philosophy, market outlook, 
portfolio strategy, and risk management.
Be supportive and remember details the user shares about themselves.""")
    
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the graph
builder = StateGraph(State)
builder.add_node("chatbot", investment_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Compile with a checkpointer for short-term memory
checkpointer = MemorySaver()
investment_graph = builder.compile(checkpointer=checkpointer)

print("Investment chatbot compiled with short-term memory (checkpointing)")

In [ ]:
# Test short-term memory within a thread
config = {"configurable": {"thread_id": "investment_thread_1"}}

# First message - introduce ourselves
response = investment_graph.invoke(
    {"messages": [HumanMessage(content="Hi! My name is Alex and I want to understand Stone Ridge's investment approach.")]},
    config
)
print("User: Hi! My name is Alex and I want to understand Stone Ridge's investment approach.")
print(f"Assistant: {response['messages'][-1].content}")
print()

In [ ]:
# Second message - test if it remembers (same thread)
response = investment_graph.invoke(
    {"messages": [HumanMessage(content="What's my name and what am I interested in learning about?")]},
    config  # Same config = same thread_id
)
print("User: What's my name and what am I interested in learning about?")
print(f"Assistant: {response['messages'][-1].content}")

In [ ]:
# New thread - it won't remember Alex!
different_config = {"configurable": {"thread_id": "investment_thread_2"}}

response = investment_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id = no memory of Alex
)
print("User (NEW thread): What's my name?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent doesn't know our name because this is a new thread!")

In [ ]:
# Inspect the state of thread 1
state = investment_graph.get_state(config)
print(f"Thread 1 has {len(state.values['messages'])} messages:")
for msg in state.values['messages']:
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

## Task 4: Long-Term Memory (InMemoryStore, namespaces)

**Long-term memory** stores information across different conversation threads. This is like remembering that your friend prefers tea over coffee - you remember it every time you meet them, regardless of what you're currently discussing.

In LangGraph, long-term memory uses:
- **Store**: A persistent key-value store
- **Namespaces**: Organize memories by user, application, or context

### Key Difference from Short-Term Memory

| Short-Term (Checkpointer) | Long-Term (Store) |
|---------------------------|-------------------|
| Scoped to a single thread | Shared across all threads |
| Automatic (messages) | Explicit (you decide what to store) |
| Conversation history | User preferences, facts, etc. |

In [ ]:
from langgraph.store.memory import InMemoryStore

# Create a store for long-term memory
store = InMemoryStore()

# Namespaces organize memories - typically by user_id and category
user_id = "user_alex"
profile_namespace = (user_id, "profile")
preferences_namespace = (user_id, "preferences")

# Store Alex's investment profile
store.put(profile_namespace, "name", {"value": "Alex"})
store.put(profile_namespace, "goals", {"primary": "long-term growth", "secondary": "income generation"})
store.put(profile_namespace, "constraints", {"risk_tolerance": "moderate", "restrictions": ["no tobacco stocks"], "esg_preference": True})
store.put(profile_namespace, "portfolio", {"size": "$500K", "horizon": "20 years", "current_allocation": "60/40 stocks/bonds"})

# Store Alex's preferences
store.put(preferences_namespace, "communication", {"style": "data-driven", "detail_level": "comprehensive"})
store.put(preferences_namespace, "reporting", {"frequency": "quarterly", "preferred_metrics": ["CAGR", "Sharpe ratio", "max drawdown"]})

print("Stored Alex's profile and preferences in long-term memory")

In [ ]:
# Retrieve specific memories
name = store.get(profile_namespace, "name")
print(f"Name: {name.value}")

goals = store.get(profile_namespace, "goals")
print(f"Goals: {goals.value}")

# List all memories in a namespace
print("\nAll profile items:")
for item in store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

In [ ]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig

# Define state with user_id for personalization
class PersonalizedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def personalized_investment_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """An investment chatbot that uses long-term memory for personalization."""
    user_id = state["user_id"]
    profile_namespace = (user_id, "profile")
    preferences_namespace = (user_id, "preferences")
    
    # Retrieve user profile from long-term memory
    profile_items = list(store.search(profile_namespace))
    pref_items = list(store.search(preferences_namespace))
    
    # Build context from profile
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items])
    pref_text = "\n".join([f"- {p.key}: {p.value}" for p in pref_items])
    
    system_msg = f"""You are an Investment Advisory Assistant. You know the following about this user:

PROFILE:
{profile_text if profile_text else 'No profile stored.'}

PREFERENCES:
{pref_text if pref_text else 'No preferences stored.'}

Use this information to personalize your responses. Be supportive and helpful."""
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the personalized graph
builder2 = StateGraph(PersonalizedState)
builder2.add_node("chatbot", personalized_investment_chatbot)
builder2.add_edge(START, "chatbot")
builder2.add_edge("chatbot", END)

# Compile with BOTH checkpointer (short-term) AND store (long-term)
personalized_graph = builder2.compile(
    checkpointer=MemorySaver(),
    store=store
)

print("Personalized graph compiled with both short-term and long-term memory")

In [ ]:
# Test the personalized chatbot - it knows Alex's profile!
config = {"configurable": {"thread_id": "personalized_thread_1"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment strategy would you recommend for me?")],
        "user_id": "user_alex"
    },
    config
)

print("User: What investment strategy would you recommend for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent knows about Alex's risk tolerance and portfolio without him mentioning it!")

In [ ]:
# Even in a NEW thread, it still knows Alex's profile
# because long-term memory is cross-thread!

new_config = {"configurable": {"thread_id": "personalized_thread_2"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="Are there any risks I should be aware of given my portfolio?")],
        "user_id": "user_alex"
    },
    new_config
)

print("User (NEW thread): Are there any risks I should be aware of given my portfolio?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: Even in a new thread, the agent knows Alex's portfolio and constraints!")

## Task 5: Message Trimming & Context Management

Long conversations can exceed the LLM's context window. LangGraph provides utilities to manage message history:

- **`trim_messages`**: Keeps only recent messages up to a token limit
- **Summarization**: Compress older messages into summaries

### Why Trim Even with 128K Context?

Even with large context windows:
1. **Cost**: More tokens = higher API costs
2. **Latency**: Larger contexts take longer to process
3. **Quality**: Models can struggle with "lost in the middle" - important info buried in long contexts
4. **Relevance**: Old messages may not be relevant to current query

In [ ]:
from langchain_core.messages import trim_messages

# Create a trimmer that keeps only recent messages
trimmer = trim_messages(
    max_tokens=500,  # Keep messages up to this token count
    strategy="last",  # Keep the most recent messages
    token_counter=llm,  # Use the LLM to count tokens
    include_system=True,  # Always keep system messages
    allow_partial=False,  # Don't cut messages in half
)

# Example: Create a long conversation
long_conversation = [
    SystemMessage(content="You are an investment advisory assistant."),
    HumanMessage(content="I want to improve my portfolio returns."),
    AIMessage(content="Great goal! Let's start with your current allocation. What does your portfolio look like?"),
    HumanMessage(content="I have about 60% stocks and 40% bonds."),
    AIMessage(content="That's a balanced allocation. For higher returns, you might consider increasing equity exposure or adding alternative investments."),
    HumanMessage(content="What about international diversification?"),
    AIMessage(content="International exposure can reduce risk through diversification. Consider allocating 20-30% to international developed and emerging markets."),
    HumanMessage(content="And alternative investments?"),
    AIMessage(content="Alternatives like reinsurance, real estate, and commodities can provide uncorrelated returns and enhance portfolio efficiency."),
    HumanMessage(content="What's the most important change I should make first?"),
]

# Trim to fit context window
trimmed = trimmer.invoke(long_conversation)
print(f"Original: {len(long_conversation)} messages")
print(f"Trimmed: {len(trimmed)} messages")
print("\nTrimmed conversation:")
for msg in trimmed:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"  {role}: {content}")

In [ ]:
# Summarization approach for longer conversations

def summarize_conversation(messages: list, max_messages: int = 6) -> list:
    """Summarize older messages to manage context length."""
    if len(messages) <= max_messages:
        return messages
    
    # Keep the system message and last few messages
    system_msg = messages[0] if isinstance(messages[0], SystemMessage) else None
    content_messages = messages[1:] if system_msg else messages
    
    if len(content_messages) <= max_messages:
        return messages
    
    old_messages = content_messages[:-max_messages+1]
    recent_messages = content_messages[-max_messages+1:]
    
    # Summarize old messages
    summary_prompt = f"""Summarize this conversation in 2-3 sentences, 
capturing key investment topics discussed and any important user information:

{chr(10).join([f'{type(m).__name__}: {m.content[:200]}' for m in old_messages])}"""
    
    summary = llm.invoke(summary_prompt)
    
    # Return: system + summary + recent messages
    result = []
    if system_msg:
        result.append(system_msg)
    result.append(SystemMessage(content=f"[Previous conversation summary: {summary.content}]"))
    result.extend(recent_messages)
    
    return result


# Test summarization
summarized = summarize_conversation(long_conversation, max_messages=4)
print(f"Summarized: {len(summarized)} messages")
print("\nSummarized conversation:")
for msg in summarized:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

---
## ❓ Question #1:

What are the trade-offs between **short-term memory** (checkpointer) vs **long-term memory** (store)? When should investment data move from short-term to long-term? Consider:
- What information should persist across sessions?
- What are the compliance implications?
- How would you decide what to promote from short-term to long-term?

##### Answer:

**Trade-offs:**

**Short-term memory (Checkpointer):**
- ✅ Automatic management of conversation history
- ✅ Scoped to individual threads - easier privacy control
- ✅ Natural for transient consultation context
- ❌ Lost when thread ends or is manually cleared
- ❌ Not queryable across conversations
- ❌ Storage grows with conversation length

**Long-term memory (Store):**
- ✅ Persists across all sessions and threads
- ✅ Queryable and searchable across user history
- ✅ Enables personalization and continuity
- ❌ Requires explicit decisions on what to store
- ❌ Storage and privacy management overhead
- ❌ Potential for stale or conflicting data

**When to move data from short-term to long-term:**

1. **User Profile Information** → Long-term
   - Risk tolerance, investment horizon, portfolio constraints
   - These are stable attributes that should persist across sessions
   
2. **Investment Goals & Constraints** → Long-term
   - Primary/secondary goals, ESG preferences, sector restrictions
   - Critical for consistent advice across consultations

3. **Important Decisions Made** → Long-term (Episodic)
   - Asset allocation changes, rebalancing decisions
   - Creates audit trail and learning opportunities

4. **Transient Market Discussion** → Short-term only
   - Current market commentary, specific price discussions
   - Time-sensitive and quickly becomes outdated

**Compliance Implications:**

- **Recordkeeping**: Financial regulations (e.g., SEC, FINRA) require maintaining records of investment advice given
- **Data Retention**: Must balance retention requirements with privacy laws (GDPR, CCPA)
- **Audit Trail**: Long-term memory should include timestamps and versioning for compliance review
- **Privacy**: User PII should be encrypted and access-controlled in long-term storage
- **Right to Erasure**: Must support deletion of user data upon request

**Decision Framework for Promotion:**

```python
def should_promote_to_long_term(data_type: str, user_action: str) -> bool:
    # Explicit user confirmations
    if user_action in ["confirmed_profile", "set_goal", "made_decision"]:
        return True
    
    # Critical constraints that affect advice
    if data_type in ["risk_tolerance", "investment_restrictions", "portfolio_size"]:
        return True
    
    # Successful outcomes worth remembering
    if data_type == "advisory_episode" and user_feedback == "positive":
        return True
    
    # Transient data stays in short-term
    if data_type in ["market_commentary", "casual_question"]:
        return False
    
    return False
```

## ❓ Question #2:

Why use message trimming with a 128K context window when the Stone Ridge investor letter is relatively small? What should **always** be preserved when trimming an investment consultation?

Consider:
- The "lost in the middle" phenomenon
- Cost and latency implications
- What user information is critical for safety (risk tolerance, constraints, etc.)

##### Answer:

**Why trim even with large context windows:**

1. **"Lost in the Middle" Phenomenon**
   - Research shows LLMs struggle to use information in the middle of very long contexts
   - Important details can be overlooked when buried in extensive conversation history
   - Shorter, focused context improves retrieval accuracy and response relevance

2. **Cost Optimization**
   - GPT-4 pricing: ~$5/1M input tokens, $15/1M output tokens
   - A 50-message conversation could be 20K+ tokens
   - Trimming to 500-2000 tokens per request saves 90%+ on costs
   - Over thousands of users and consultations, savings compound significantly

3. **Latency Reduction**
   - First-token latency matters for user experience
   - Shorter contexts = faster responses = better UX

4. **Quality > Quantity**
   - Recent context is usually most relevant
   - Long-ago messages may contain outdated or conflicting information
   - Focused context helps the model stay on topic

**What must ALWAYS be preserved:**

1. **System Instructions** (`include_system=True`)
   - Core advisory guidelines and compliance requirements
   - Procedural memory (self-improving instructions)
   - Non-negotiable behavior boundaries

2. **User Risk Profile & Constraints** (via long-term memory)
   - Risk tolerance level (conservative/moderate/aggressive)
   - Investment restrictions (e.g., "no tobacco stocks", "ESG only")
   - Legal constraints (accredited investor status, jurisdiction)
   - Portfolio size and investment horizon
   - **Why**: Giving inappropriate advice for risk profile is dangerous

3. **Previously Established Goals & Context**
   - Investment objectives stated in current thread
   - Important decisions made in current consultation
   - **Why**: Contradicting previous statements in same conversation erodes trust

4. **Compliance-Critical Information**
   - Disclaimers that have been provided
   - Acknowledgment that advice is not a guarantee
   - Regulatory disclosures

**Trimming Strategy:**

```python
def safe_investment_trimmer(messages, max_tokens=2000):
    # Extract critical information before trimming
    risk_profile = extract_from_long_term_memory()
    current_goals = extract_goals_from_conversation(messages)
    
    # Build system message with critical context
    system_msg = f"""Investment Advisory Assistant
    
    USER PROFILE (ALWAYS REMEMBER):
    - Risk Tolerance: {risk_profile.risk_tolerance}
    - Investment Restrictions: {risk_profile.restrictions}
    - Portfolio Size: {risk_profile.portfolio_size}
    - Investment Horizon: {risk_profile.horizon}
    
    CURRENT SESSION GOALS:
    {current_goals}
    
    [Standard compliance disclaimers...]
    """
    
    # Trim conversation but keep system message
    trimmed = trim_messages(
        messages,
        max_tokens=max_tokens,
        include_system=True,
        strategy="last"
    )
    
    return [SystemMessage(content=system_msg)] + trimmed[1:]
```

**Alternative: Semantic Summarization**
- Instead of dropping old messages entirely, summarize them
- Preserves key decisions and context while reducing tokens
- Example: "Earlier in conversation: user expressed concern about market volatility and preference for defensive positioning"

---
## 🏗️ Activity #1: Store & Retrieve User Investment Profile

Build a complete investment profile system that:
1. Defines an investment profile schema (name, risk tolerance, portfolio size, investment horizon, restrictions, goals)
2. Creates functions to store and retrieve profile data
3. Builds a personalized investment agent that uses the profile
4. Tests that different users get different advice

### Requirements:
- Define at least 5 profile attributes
- Support multiple users with different profiles
- Agent should reference profile data in responses

In [ ]:
from typing import Literal, Optional, List
from datetime import datetime

class InvestmentProfile:
    """Schema for user investment profile."""
    name: str
    risk_tolerance: Literal["conservative", "moderate", "aggressive"]
    portfolio_size: str
    investment_horizon: str  # e.g., "5 years", "20 years", "indefinite"
    restrictions: List[str]  # e.g., ["no tobacco", "no weapons"]
    goals: dict  # {"primary": "...", "secondary": "..."}
    preferred_asset_classes: List[str]
    esg_preference: bool
    accredited_investor: bool
    annual_income: Optional[str]


def store_investment_profile(store, user_id: str, profile: dict):
    """Store a user's investment profile."""
    namespace = (user_id, "profile")
    
    # Store each profile attribute separately for easier querying
    for key, value in profile.items():
        store.put(namespace, key, {"value": value, "updated_at": datetime.now().isoformat()})
    
    print(f"Stored investment profile for {user_id}")


def get_investment_profile(store, user_id: str) -> dict:
    """Retrieve a user's investment profile."""
    namespace = (user_id, "profile")
    
    # Retrieve all profile items
    items = list(store.search(namespace))
    
    if not items:
        return {}
    
    # Reconstruct profile dictionary
    profile = {}
    for item in items:
        profile[item.key] = item.value["value"]
    
    return profile


store_activity1 = InMemoryStore()

# Profile 1: Conservative retiree
profile_sarah = {
    "name": "Sarah Chen",
    "risk_tolerance": "conservative",
    "portfolio_size": "$2.5M",
    "investment_horizon": "indefinite (retired)",
    "restrictions": ["no tobacco", "no weapons", "ESG preferred"],
    "goals": {
        "primary": "income generation and capital preservation",
        "secondary": "modest growth to outpace inflation"
    },
    "preferred_asset_classes": ["bonds", "dividend stocks", "REITs"],
    "esg_preference": True,
    "accredited_investor": True,
    "annual_income": "$150K (from portfolio)"
}

# Profile 2: Aggressive young professional
profile_david = {
    "name": "David Kim",
    "risk_tolerance": "aggressive",
    "portfolio_size": "$200K",
    "investment_horizon": "30 years",
    "restrictions": [],
    "goals": {
        "primary": "maximize long-term capital appreciation",
        "secondary": "explore alternative investments"
    },
    "preferred_asset_classes": ["growth stocks", "crypto", "venture capital", "alternatives"],
    "esg_preference": False,
    "accredited_investor": False,
    "annual_income": "$180K (salary)"
}

store_investment_profile(store_activity1, "user_sarah", profile_sarah)
store_investment_profile(store_activity1, "user_david", profile_david)

print("\n" + "="*60)
print("Created two user profiles:")
print(f"1. Sarah Chen - Conservative retiree")
print(f"2. David Kim - Aggressive young professional")


class ProfileState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def profile_based_advisor(state: ProfileState, config: RunnableConfig, *, store: BaseStore):
    """Investment advisor that personalizes based on user profile."""
    user_id = state["user_id"]
    profile = get_investment_profile(store, user_id)
    
    if not profile:
        system_msg = "You are an Investment Advisory Assistant. Ask the user about their investment profile first."
    else:
        # Build detailed profile context
        profile_text = f"""
User: {profile.get('name', 'Unknown')}
Risk Tolerance: {profile.get('risk_tolerance', 'Unknown')}
Portfolio Size: {profile.get('portfolio_size', 'Unknown')}
Investment Horizon: {profile.get('investment_horizon', 'Unknown')}
Investment Goals: {profile.get('goals', {})}
Restrictions: {', '.join(profile.get('restrictions', [])) if profile.get('restrictions') else 'None'}
Preferred Asset Classes: {', '.join(profile.get('preferred_asset_classes', []))}
ESG Preference: {'Yes' if profile.get('esg_preference') else 'No'}
Accredited Investor: {'Yes' if profile.get('accredited_investor') else 'No'}
"""
        
        system_msg = f"""You are an Investment Advisory Assistant. You have access to this user's profile:

{profile_text}

IMPORTANT GUIDELINES:
- Tailor ALL recommendations to their risk tolerance level
- Respect their investment restrictions absolutely
- Align advice with their stated goals and time horizon
- Only suggest accredited-investor-only investments if they qualify
- If they prefer ESG, prioritize sustainable investment options
- Reference their preferred asset classes when making suggestions

Always provide personalized, profile-appropriate advice."""
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the graph
builder_activity1 = StateGraph(ProfileState)
builder_activity1.add_node("advisor", profile_based_advisor)
builder_activity1.add_edge(START, "advisor")
builder_activity1.add_edge("advisor", END)

profile_advisor_graph = builder_activity1.compile(
    checkpointer=MemorySaver(),
    store=store_activity1
)

print("\n" + "="*60)
print("Profile-based advisor agent ready!")


print("\n" + "="*60)
print("Testing with Sarah (conservative retiree)...")
print("="*60)

response_sarah = profile_advisor_graph.invoke(
    {
        "messages": [HumanMessage(content="I'm interested in alternative investments. What would you recommend for me?")],
        "user_id": "user_sarah"
    },
    {"configurable": {"thread_id": "sarah_thread_1"}}
)

print(f"\nSarah asks: I'm interested in alternative investments. What would you recommend for me?")
print(f"\nAdvisor response:\n{response_sarah['messages'][-1].content}")

print("\n" + "="*60)
print("Testing with David (aggressive young professional)...")
print("="*60)

response_david = profile_advisor_graph.invoke(
    {
        "messages": [HumanMessage(content="I'm interested in alternative investments. What would you recommend for me?")],
        "user_id": "user_david"
    },
    {"configurable": {"thread_id": "david_thread_1"}}
)

print(f"\nDavid asks: I'm interested in alternative investments. What would you recommend for me?")
print(f"\nAdvisor response:\n{response_david['messages'][-1].content}")

print("\n" + "="*60)
print("COMPARISON:")
print("Notice how the SAME question gets DIFFERENT advice based on:")
print("- Sarah: Conservative, retired, ESG-focused → safer alternatives")
print("- David: Aggressive, 30-year horizon → higher-risk alternatives")
print("="*60)

---
# 🤝 Breakout Room #2
## Advanced Memory & Integration

## Task 6: Semantic Memory (Embeddings + Search)

**Semantic memory** stores facts and retrieves them based on *meaning* rather than exact matches. This is like how you might remember "that fund with the great risk-adjusted returns" even if you can't remember its exact name.

In LangGraph, semantic memory uses:
- **Store with embeddings**: Converts text to vectors for similarity search
- **`store.search()`**: Finds relevant memories by semantic similarity

### How It Works

```
User asks: "What helps with portfolio diversification?"
         ↓
Query embedded → [0.2, 0.8, 0.1, ...]
         ↓
Compare with stored investment facts:
  - "Uncorrelated assets reduce portfolio risk" → 0.92 similarity ✓
  - "Rebalancing maintains target allocations" → 0.35 similarity
         ↓
Return: "Uncorrelated assets reduce portfolio risk"
```

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Create embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a store with semantic search enabled
semantic_store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,  # Dimension of text-embedding-3-small
    }
)

print("Semantic memory store created with embedding support")

In [ ]:
# Store various investment facts as semantic memories
namespace = ("investment", "facts")

investment_facts = [
    ("fact_1", {"text": "Diversification across uncorrelated assets can reduce portfolio risk without sacrificing returns"}),
    ("fact_2", {"text": "Stone Ridge focuses on alternative risk premiums including reinsurance and longevity risk"}),
    ("fact_3", {"text": "Tail risk hedging provides insurance against extreme market downturns"}),
    ("fact_4", {"text": "A long-term investment horizon allows investors to capture illiquidity premiums"}),
    ("fact_5", {"text": "Factor investing targets specific drivers of return such as value, momentum, and quality"}),
    ("fact_6", {"text": "Rebalancing portfolios periodically helps maintain target risk levels"}),
    ("fact_7", {"text": "Alternative investments like reinsurance have low correlation with traditional stock and bond markets"}),
    ("fact_8", {"text": "Systematic risk management frameworks help identify and mitigate portfolio vulnerabilities"}),
]

for key, value in investment_facts:
    semantic_store.put(namespace, key, value)

print(f"Stored {len(investment_facts)} investment facts in semantic memory")

In [ ]:
# Search semantically - notice we don't need exact matches!

queries = [
    "How can I protect my portfolio from a market crash?",
    "What alternative investments should I consider?",
    "How should I think about risk in my portfolio?",
    "What is Stone Ridge's investment approach?",
]

for query in queries:
    print(f"\nQuery: {query}")
    results = semantic_store.search(namespace, query=query, limit=2)
    for r in results:
        print(f"   {r.value['text']} (score: {r.score:.3f})")

## Task 7: Building Semantic Investment Knowledge Base

Let's load the Stone Ridge 2025 Investor Letter and create a semantic knowledge base that our agent can search.

This is similar to RAG from Module 4, but now using LangGraph's Store API instead of a separate vector database.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk the investment document
loader = PyMuPDFLoader("data/Stone Ridge 2025 Investor Letter.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)

print(f"Loaded and split into {len(chunks)} chunks")
print(f"\nSample chunk:\n{chunks[0].page_content[:200]}...")

In [ ]:
# Store chunks in semantic memory
knowledge_namespace = ("investment", "knowledge")

for i, chunk in enumerate(chunks):
    semantic_store.put(
        knowledge_namespace,
        f"chunk_{i}",
        {"text": chunk.page_content, "source": "Stone Ridge 2025 Investor Letter.pdf"}
    )

print(f"Stored {len(chunks)} chunks in semantic knowledge base")

In [ ]:
# Build a semantic search investment chatbot

class SemanticState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def semantic_investment_chatbot(state: SemanticState, config: RunnableConfig, *, store: BaseStore):
    """An investment chatbot that retrieves relevant facts using semantic search."""
    user_message = state["messages"][-1].content
    
    # Search for relevant knowledge
    knowledge_results = store.search(
        ("investment", "knowledge"),
        query=user_message,
        limit=3
    )
    
    # Build context from retrieved knowledge
    if knowledge_results:
        knowledge_text = "\n\n".join([f"- {r.value['text']}" for r in knowledge_results])
        system_msg = f"""You are an Investment Advisory Assistant with access to the Stone Ridge investor letter knowledge base.

Relevant information from your knowledge base:
{knowledge_text}

Use this information to answer the user's question. If the information doesn't directly answer their question, use your general knowledge but mention what you found."""
    else:
        system_msg = "You are an Investment Advisory Assistant. Answer investment questions helpfully."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build and compile
builder3 = StateGraph(SemanticState)
builder3.add_node("chatbot", semantic_investment_chatbot)
builder3.add_edge(START, "chatbot")
builder3.add_edge("chatbot", END)

semantic_graph = builder3.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Semantic investment chatbot ready")

In [ ]:
# Test semantic retrieval
config = {"configurable": {"thread_id": "semantic_thread_1"}}

questions = [
    "What is Stone Ridge's view on the current market environment?",
    "How does Stone Ridge approach risk management?",
    "What is Stone Ridge's investment philosophy?",
]

for q in questions:
    response = semantic_graph.invoke(
        {"messages": [HumanMessage(content=q)], "user_id": "test_user"},
        config
    )
    print(f"\nUser: {q}")
    print(f"Assistant: {response['messages'][-1].content[:500]}...")

## Task 8: Episodic Memory (Few-Shot Learning)

**Episodic memory** stores past experiences and uses them as examples for future tasks. In humans, this is like remembering the first time you provided a successful investment recommendation and using that experience to guide future interactions.

For AI agents, episodic memory is often implemented as **few-shot examples** - showing the model examples of correct behavior so it can learn patterns.

### The CoALA Framework Perspective

> "Facts can be written to semantic memory, whereas **experiences** can be written to episodic memory."

Key insight: Sometimes it's easier to **show** than **tell**. LLMs learn well from examples!

In [ ]:
# Create episodic memory with past successful interactions
episodic_namespace = ("agent", "episodes")

successful_episodes = [
    {
        "situation": "User asked about portfolio diversification with concentrated holdings",
        "input": "I have most of my portfolio in tech stocks. How should I diversify?",
        "output": """For a concentrated tech portfolio, I'd recommend diversifying across several dimensions:

1. **Asset classes** - Add bonds, real estate (REITs), and alternatives
2. **Geographies** - Include international developed and emerging markets
3. **Sectors** - Shift allocation toward healthcare, consumer staples, and industrials
4. **Alternative risk premiums** - Consider reinsurance or longevity-linked investments

Start by gradually rebalancing 5-10% per quarter to avoid tax impacts. Would you like a specific rebalancing plan?""",
        "feedback": "User appreciated the structured approach and specific action items"
    },
    {
        "situation": "User concerned about market downturn impact on retirement portfolio",
        "input": "I'm 10 years from retirement and worried about a market crash. What should I do?",
        "output": """Market timing is difficult, but there are proven strategies to protect near-retirement portfolios:

**Risk reduction:**
- Gradually shift to a more conservative allocation (e.g., 50/50 stocks/bonds)
- Consider tail risk hedging for catastrophic downside protection

**Income stability:**
- Build a 2-3 year cash buffer for living expenses
- Add dividend-paying stocks and investment-grade bonds

**Stay invested:**
- Keep some equity exposure for growth to combat inflation
- Focus on quality companies with strong balance sheets

Which aspect would you like to explore in more detail?""",
        "feedback": "User found the balanced approach reassuring and actionable"
    },
    {
        "situation": "User asking about alternative investments",
        "input": "What are alternative investments and should I have them in my portfolio?",
        "output": """Alternative investments are assets beyond traditional stocks and bonds. They can add diversification and unique return sources:

**Common alternatives:**
- Real estate (REITs, direct property)
- Private equity and venture capital
- Hedge funds and managed futures
- Reinsurance and catastrophe bonds
- Commodities and natural resources

**Benefits:**
- Low correlation with traditional markets
- Access to different risk premiums
- Potential for higher risk-adjusted returns

**Considerations:**
- Often less liquid than public markets
- May require higher minimum investments
- Due diligence is critical

A typical allocation might be 10-20% of your portfolio. What's your current portfolio size and investment horizon?""",
        "feedback": "User valued the comprehensive overview without being overwhelming"
    },
]

for i, episode in enumerate(successful_episodes):
    semantic_store.put(
        episodic_namespace,
        f"episode_{i}",
        {
            "text": episode["situation"],  # Used for semantic search
            **episode
        }
    )

print(f"Stored {len(successful_episodes)} episodic memories (past successful interactions)")

In [ ]:
class EpisodicState(TypedDict):
    messages: Annotated[list, add_messages]


def episodic_investment_chatbot(state: EpisodicState, config: RunnableConfig, *, store: BaseStore):
    """A chatbot that learns from past successful interactions."""
    user_question = state["messages"][-1].content
    
    # Search for similar past experiences
    similar_episodes = store.search(
        ("agent", "episodes"),
        query=user_question,
        limit=1
    )
    
    # Build few-shot examples from past episodes
    if similar_episodes:
        episode = similar_episodes[0].value
        few_shot_example = f"""Here's an example of a similar investment question I handled well:

User asked: {episode['input']}

My response was:
{episode['output']}

The user feedback was: {episode['feedback']}

Use this as inspiration for the style, structure, and tone of your response, but tailor it to the current question."""
        
        system_msg = f"""You are an Investment Advisory Assistant. Learn from your past successes:

{few_shot_example}"""
    else:
        system_msg = "You are an Investment Advisory Assistant. Be helpful, specific, and supportive."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the episodic memory graph
builder4 = StateGraph(EpisodicState)
builder4.add_node("chatbot", episodic_investment_chatbot)
builder4.add_edge(START, "chatbot")
builder4.add_edge("chatbot", END)

episodic_graph = builder4.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Episodic memory chatbot ready")

In [ ]:
# Test episodic memory - similar question to stored episode
config = {"configurable": {"thread_id": "episodic_thread_1"}}

response = episodic_graph.invoke(
    {"messages": [HumanMessage(content="I'm thinking about adding some alternative investments to my portfolio. What should I consider?")]},
    config
)

print("User: I'm thinking about adding some alternative investments to my portfolio. What should I consider?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The response structure mirrors the successful alternatives episode!")

## Task 9: Procedural Memory (Self-Improving Agent)

**Procedural memory** stores the rules and instructions that guide behavior. In humans, this is like knowing *how* to give good advice - it's internalized knowledge about performing tasks.

For AI agents, procedural memory often means **self-modifying prompts**. The agent can:
1. Store its current instructions in the memory store
2. Reflect on feedback from interactions
3. Update its own instructions to improve

### The Reflection Pattern

```
User feedback: "Your advice is too long and complicated"
         ↓
Agent reflects on current instructions
         ↓
Agent updates instructions: "Keep advice concise and actionable"
         ↓
Future responses use updated instructions
```

In [ ]:
# Initialize procedural memory with base instructions
procedural_namespace = ("agent", "instructions")

initial_instructions = """You are an Investment Advisory Assistant.

Guidelines:
- Be objective and data-driven in your analysis
- Provide evidence-based investment information
- Ask clarifying questions about risk tolerance and investment goals
- Present balanced perspectives on investment decisions
- Always note that past performance doesn't guarantee future results"""

semantic_store.put(
    procedural_namespace,
    "investment_assistant",
    {"instructions": initial_instructions, "version": 1}
)

print("Initialized procedural memory with base instructions")
print(f"\nCurrent Instructions (v1):\n{initial_instructions}")

In [ ]:
class ProceduralState(TypedDict):
    messages: Annotated[list, add_messages]
    feedback: str  # Optional feedback from user


def get_instructions(store: BaseStore) -> tuple[str, int]:
    """Retrieve current instructions from procedural memory."""
    item = store.get(("agent", "instructions"), "investment_assistant")
    if item is None:
        return "You are a helpful investment advisory assistant.", 0
    return item.value["instructions"], item.value["version"]


def procedural_assistant_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Respond using current procedural instructions."""
    instructions, version = get_instructions(store)
    
    messages = [SystemMessage(content=instructions)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def reflection_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Reflect on feedback and update instructions if needed."""
    feedback = state.get("feedback", "")
    
    if not feedback:
        return {}  # No feedback, no update needed
    
    # Get current instructions
    current_instructions, version = get_instructions(store)
    
    # Ask the LLM to reflect and improve instructions
    reflection_prompt = f"""You are improving an investment advisory assistant's instructions based on user feedback.

Current Instructions:
{current_instructions}

User Feedback:
{feedback}

Based on this feedback, provide improved instructions. Keep the same general format but incorporate the feedback.
Only output the new instructions, nothing else."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    new_instructions = response.content
    
    # Update procedural memory with new instructions
    store.put(
        ("agent", "instructions"),
        "investment_assistant",
        {"instructions": new_instructions, "version": version + 1}
    )
    
    print(f"\nInstructions updated to version {version + 1}")
    return {}


def should_reflect(state: ProceduralState) -> str:
    """Decide whether to reflect on feedback."""
    if state.get("feedback"):
        return "reflect"
    return "end"


# Build the procedural memory graph
builder5 = StateGraph(ProceduralState)
builder5.add_node("assistant", procedural_assistant_node)
builder5.add_node("reflect", reflection_node)

builder5.add_edge(START, "assistant")
builder5.add_conditional_edges("assistant", should_reflect, {"reflect": "reflect", "end": END})
builder5.add_edge("reflect", END)

procedural_graph = builder5.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Procedural memory graph ready (with self-improvement capability)")

In [ ]:
# Test with initial instructions
config = {"configurable": {"thread_id": "procedural_thread_1"}}

response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How should I think about portfolio risk?")],
        "feedback": ""  # No feedback yet
    },
    config
)

print("User: How should I think about portfolio risk?")
print(f"\nAssistant (v1 instructions):\n{response['messages'][-1].content}")

In [ ]:
# Now provide feedback - the agent will update its own instructions!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How should I think about portfolio risk?")],
        "feedback": "Your responses are too long. Please be more concise and give me 3 actionable insights maximum."
    },
    {"configurable": {"thread_id": "procedural_thread_2"}}
)

In [ ]:
# Check the updated instructions
new_instructions, version = get_instructions(semantic_store)
print(f"Updated Instructions (v{version}):\n")
print(new_instructions)

In [ ]:
# Test with updated instructions - should be more concise now!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment opportunities should I consider in the current market?")],
        "feedback": ""  # No feedback this time
    },
    {"configurable": {"thread_id": "procedural_thread_3"}}
)

print(f"User: What investment opportunities should I consider in the current market?")
print(f"\nAssistant (v{version} instructions - after feedback):")
print(response['messages'][-1].content)
print("\nNotice: The response should now be more concise based on the feedback!")

## Task 10: Unified Investment Memory Agent

Now let's combine **all 5 memory types** into a unified investment advisory agent:

1. **Short-term**: Remembers current conversation (checkpointer)
2. **Long-term**: Stores user profile across sessions (store + namespace)
3. **Semantic**: Retrieves relevant investment knowledge (store + embeddings)
4. **Episodic**: Uses past successful interactions as examples (store + search)
5. **Procedural**: Adapts behavior based on feedback (store + reflection)

### Memory Retrieval Flow

```
User Query: "What investment strategy suits my risk profile?"
              │
              ▼
┌─────────────────────────────────────────────────┐
│  1. PROCEDURAL: Get current instructions         │
│  2. LONG-TERM: Load user profile (constraints)   │
│  3. SEMANTIC: Search investment knowledge        │
│  4. EPISODIC: Find similar past interactions     │
│  5. SHORT-TERM: Include conversation history     │
└─────────────────────────────────────────────────┘
              │
              ▼
        Generate personalized, informed response
```

In [ ]:
class UnifiedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    feedback: str


def unified_investment_assistant(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """An assistant that uses all five memory types."""
    user_id = state["user_id"]
    user_message = state["messages"][-1].content
    
    # 1. PROCEDURAL: Get current instructions
    instructions_item = store.get(("agent", "instructions"), "investment_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful investment advisory assistant."
    
    # 2. LONG-TERM: Get user profile
    profile_items = list(store.search((user_id, "profile")))
    pref_items = list(store.search((user_id, "preferences")))
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items]) if profile_items else "No profile stored."
    
    # 3. SEMANTIC: Search for relevant knowledge
    relevant_knowledge = store.search(("investment", "knowledge"), query=user_message, limit=2)
    knowledge_text = "\n".join([f"- {r.value['text'][:200]}..." for r in relevant_knowledge]) if relevant_knowledge else "No specific knowledge found."
    
    # 4. EPISODIC: Find similar past interactions
    similar_episodes = store.search(("agent", "episodes"), query=user_message, limit=1)
    if similar_episodes:
        ep = similar_episodes[0].value
        episode_text = f"Similar past interaction:\nUser: {ep.get('input', 'N/A')}\nResponse style: {ep.get('feedback', 'N/A')}"
    else:
        episode_text = "No similar past interactions found."
    
    # Build comprehensive system message
    system_message = f"""{base_instructions}

=== USER PROFILE ===
{profile_text}

=== RELEVANT INVESTMENT KNOWLEDGE ===
{knowledge_text}

=== LEARNING FROM EXPERIENCE ===
{episode_text}

Use all of this context to provide the best possible personalized response."""
    
    # 5. SHORT-TERM: Full conversation history is automatically managed by the checkpointer
    # Use summarization for long conversations
    trimmed_messages = summarize_conversation(state["messages"], max_messages=6)
    
    messages = [SystemMessage(content=system_message)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": [response]}


def unified_feedback_node(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """Update procedural memory based on feedback."""
    feedback = state.get("feedback", "")
    if not feedback:
        return {}
    
    item = store.get(("agent", "instructions"), "investment_assistant")
    if item is None:
        return {}
    
    current = item.value
    reflection_prompt = f"""Update these instructions based on feedback:

Current: {current['instructions']}
Feedback: {feedback}

Output only the updated instructions."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    store.put(
        ("agent", "instructions"),
        "investment_assistant",
        {"instructions": response.content, "version": current["version"] + 1}
    )
    print(f"Procedural memory updated to v{current['version'] + 1}")
    return {}


def unified_route(state: UnifiedState) -> str:
    return "feedback" if state.get("feedback") else "end"


# Build the unified graph
unified_builder = StateGraph(UnifiedState)
unified_builder.add_node("assistant", unified_investment_assistant)
unified_builder.add_node("feedback", unified_feedback_node)

unified_builder.add_edge(START, "assistant")
unified_builder.add_conditional_edges("assistant", unified_route, {"feedback": "feedback", "end": END})
unified_builder.add_edge("feedback", END)

# Compile with both checkpointer (short-term) and store (all other memory types)
unified_graph = unified_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Unified investment assistant ready with all 5 memory types!")

In [ ]:
# Test the unified assistant
config = {"configurable": {"thread_id": "unified_thread_1"}}

# First interaction - should use semantic + long-term + episodic memory
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="What investment strategy would you recommend given my profile?")],
        "user_id": "user_alex",  # Alex has moderate risk tolerance and ESG preferences!
        "feedback": ""
    },
    config
)

print("User: What investment strategy would you recommend given my profile?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\n" + "="*60)
print("Memory types used:")
print("  Long-term: Knows Alex's risk tolerance, portfolio, and ESG preferences")
print("  Semantic: Retrieved investment knowledge from Stone Ridge letter")
print("  Episodic: May use similar advisory episode as reference")
print("  Procedural: Following current instructions")
print("  Short-term: Will remember this in follow-up questions")

In [ ]:
# Follow-up question (tests short-term memory)
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you tell me more about the alternative investments you mentioned?")],
        "user_id": "user_alex",
        "feedback": ""
    },
    config  # Same thread
)

print("User: Can you tell me more about the alternative investments you mentioned?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The agent remembers the context from the previous message!")

---
## ❓ Question #3:

How would you decide what constitutes a **"successful" investment advisory interaction** worth storing as an episode? What metadata should you store alongside the episode?

Consider:
- Explicit feedback (thumbs up) vs implicit signals
- User engagement (did they ask follow-up questions?)
- Objective outcomes vs subjective satisfaction
- Privacy implications of storing interaction data

##### Answer:

**Defining "Successful" Investment Interactions:**

**Explicit Signals (Strongest):**
1. **Direct Positive Feedback**
   - User clicks "helpful" or gives thumbs up
   - User explicitly says "this was exactly what I needed"
   - User rates the interaction 4-5 stars

2. **Actionable Follow-Through**
   - User implements the advice (tracked through subsequent conversations)
   - User returns to report positive outcomes
   - User schedules a follow-up consultation

**Implicit Signals (Moderate):**
3. **Engagement Patterns**
   - User asks thoughtful follow-up questions (indicates value)
   - Longer session duration (suggests deep engagement)
   - User bookmarks or saves the conversation
   - User asks to schedule another consultation soon

4. **Behavioral Indicators**
   - User references this advice in future conversations
   - No immediate pushback or confusion
   - Clear understanding demonstrated in follow-ups

**Anti-Patterns (NOT Successful):**
- User asks for clarification repeatedly (advice was unclear)
- User disputes or challenges the recommendation
- User leaves mid-conversation
- No follow-up engagement at all

**Episodic Memory Metadata Schema:**

```python
{
    "episode_id": "uuid-string",
    "timestamp": "2025-01-15T14:30:00Z",
    "user_id": "user_123",  # Hashed/anonymized
    
    # Context
    "situation": "User with concentrated tech holdings seeking diversification",
    "user_profile_snapshot": {
        "risk_tolerance": "moderate",
        "portfolio_size": "$500K",
        "investment_horizon": "15 years"
    },
    
    # Interaction
    "user_query": "I have most of my portfolio in tech stocks. How should I diversify?",
    "agent_response": "[full response text]",
    "response_structure": "3-part framework: asset classes, geographies, action plan",
    
    # Success Indicators
    "success_score": 0.92,  # Composite score
    "explicit_feedback": {
        "thumbs_up": True,
        "rating": 5,
        "comment": "Very helpful and actionable"
    },
    "implicit_signals": {
        "follow_up_questions": 2,
        "session_duration_seconds": 480,
        "implemented_advice": True  # from later conversation
    },
    
    # Learning
    "what_worked": "Structured 3-part framework, specific percentages, phased approach",
    "advisor_notes": "User responded well to visual breakdown and concrete action steps",
    
    # Compliance & Privacy
    "anonymized": True,
    "contains_pii": False,
    "retention_days": 730,  # 2 years
    "user_consent_for_storage": True,
    
    # Embeddings for Semantic Search
    "embedding": [0.1, 0.2, ...],  # 1536-dim vector
    "tags": ["diversification", "tech-heavy-portfolio", "moderate-risk"]
}
```

**Success Scoring Formula:**

```python
def calculate_success_score(episode_data: dict) -> float:
    score = 0.0
    
    # Explicit feedback (60% weight)
    if episode_data.get("explicit_feedback"):
        if episode_data["explicit_feedback"].get("thumbs_up"):
            score += 0.3
        rating = episode_data["explicit_feedback"].get("rating", 0)
        score += (rating / 5.0) * 0.3  # Normalize to 0-0.3
    
    # Engagement (20% weight)
    follow_ups = min(episode_data["implicit_signals"].get("follow_up_questions", 0), 5)
    score += (follow_ups / 5.0) * 0.1
    
    session_duration = min(episode_data["implicit_signals"].get("session_duration_seconds", 0), 600)
    score += (session_duration / 600.0) * 0.1
    
    # Implementation (20% weight)
    if episode_data["implicit_signals"].get("implemented_advice"):
        score += 0.2
    
    return min(score, 1.0)  # Cap at 1.0


# Store only high-quality episodes
if calculate_success_score(episode_data) >= 0.7:
    store.put(("agent", "episodes"), episode_id, episode_data)
```

**Privacy Considerations:**

1. **Anonymization**
   - Hash user IDs before storage
   - Remove or generalize specific portfolio values
   - Strip out any mentioned names, addresses, or account numbers

2. **User Consent**
   - Explicitly ask users to opt-in to "help improve our advisory service"
   - Provide clear data retention and usage policies
   - Allow users to request deletion of their episodes

3. **Regulatory Compliance**
   - Ensure episodes don't violate fiduciary duty
   - Maintain audit trail for compliance review
   - Store episodes separately from user PII

4. **Retention Policy**
   - Keep episodes for limited time (e.g., 2 years)
   - Automatically purge low-value episodes quarterly
   - Archive successful episodes for model fine-tuning

**When NOT to Store:**
- Conversations involving errors or bad advice
- Discussions of specific securities (insider trading concerns)
- Interactions where user was frustrated or confused
- Any conversation containing compliance violations

## ❓ Question #4:

For a **production investment advisory assistant**, which memory types need persistent storage (PostgreSQL) vs in-memory? How would you handle memory across multiple agent instances (e.g., Market Outlook Agent, Strategy Agent, Risk Management Agent)?

Consider:
- Which memories are user-specific vs shared?
- Consistency requirements across agents
- Memory expiration and cleanup policies
- Namespace strategy for multi-agent systems

##### Answer:

**Memory Storage Architecture:**

| Memory Type | Storage | Rationale | TTL Policy |
|-------------|---------|-----------|------------|
| **Short-term** | PostgreSQL (`PostgresSaver`) | Must survive restarts, user expects continuity | 30 days of inactivity |
| **Long-term (User Profiles)** | PostgreSQL (`PostgresStore`) | Critical user data, must never lose | Never expire (unless user deletion) |
| **Semantic (Knowledge Base)** | PostgreSQL + pgvector | Shared across all users, needs vector search | Update on new content |
| **Episodic (Past Interactions)** | PostgreSQL | Learning resource, compliance audit trail | 2 years |
| **Procedural (Instructions)** | PostgreSQL + Redis cache | Shared across users, needs versioning | Never expire, version history |

**Why PostgreSQL vs In-Memory:**

**Use PostgreSQL for:**
- User-specific data (profiles, conversation history)
- Compliance and audit requirements
- Data that must survive server restarts
- Multi-instance deployment (shared state)

**Use In-Memory (Redis) for:**
- Hot cache of frequently accessed data
- Temporary session data (< 5 minutes)
- Rate limiting and request tracking
- Real-time feature flags

**Multi-Agent Namespace Strategy:**

```python
# Namespace Format: (scope, category, [agent_name])

# ===== SHARED MEMORIES (Cross-Agent) =====

# User Profiles - accessible by ALL agents
(user_id, "profile")              # User: risk tolerance, goals, constraints
(user_id, "preferences")          # User: communication style, reporting frequency

# Investment Knowledge Base - accessible by ALL agents
("investment", "knowledge")       # Semantic: Stone Ridge documents, market data
("investment", "facts")           # Semantic: General investment principles

# ===== AGENT-SPECIFIC MEMORIES =====

# Market Outlook Agent
("market_agent", "instructions")       # Procedural: Market agent's system prompt
("market_agent", "episodes")           # Episodic: Successful market analyses
(user_id, "market_agent", "history")   # Short-term: User's market discussions

# Strategy Agent  
("strategy_agent", "instructions")     # Procedural: Strategy agent's system prompt
("strategy_agent", "episodes")         # Episodic: Successful strategy consultations
(user_id, "strategy_agent", "history") # Short-term: User's strategy discussions

# Risk Management Agent
("risk_agent", "instructions")         # Procedural: Risk agent's system prompt
("risk_agent", "episodes")             # Episodic: Successful risk assessments
(user_id, "risk_agent", "history")     # Short-term: User's risk discussions

# ===== CROSS-AGENT LEARNING =====
("agents", "shared_episodes")          # Best practices any agent can learn from
```

**Cross-Agent Memory Sharing Example:**

```python
class MultiAgentMemoryManager:
    def __init__(self, store: PostgresStore):
        self.store = store
    
    def get_user_context(self, user_id: str) -> dict:
        """Get shared user context for ANY agent."""
        # All agents should know the user profile
        profile = list(self.store.search((user_id, "profile")))
        preferences = list(self.store.search((user_id, "preferences")))
        
        return {
            "profile": {p.key: p.value for p in profile},
            "preferences": {p.key: p.value for p in preferences}
        }
    
    def get_agent_instructions(self, agent_name: str, version: int = None) -> str:
        """Get agent-specific procedural memory."""
        item = self.store.get((f"{agent_name}", "instructions"), "current")
        if version is not None:
            item = self.store.get((f"{agent_name}", "instructions"), f"v{version}")
        return item.value["instructions"]
    
    def search_cross_agent_episodes(self, query: str, limit: int = 3) -> list:
        """Search episodes from ALL agents for learning."""
        # Search shared episodes
        shared = self.store.search(("agents", "shared_episodes"), query=query, limit=limit)
        
        # Could also search agent-specific episodes
        market = self.store.search(("market_agent", "episodes"), query=query, limit=1)
        strategy = self.store.search(("strategy_agent", "episodes"), query=query, limit=1)
        risk = self.store.search(("risk_agent", "episodes"), query=query, limit=1)
        
        return list(shared) + list(market) + list(strategy) + list(risk)
    
    def log_agent_interaction(self, user_id: str, agent_name: str, 
                            query: str, response: str, success_score: float):
        """Log interaction for both agent-specific and potentially shared learning."""
        episode_id = str(uuid4())
        
        # Store in agent-specific namespace
        self.store.put(
            (f"{agent_name}", "episodes"),
            episode_id,
            {
                "user_id": user_id,  # anonymized
                "query": query,
                "response": response,
                "success_score": success_score,
                "timestamp": datetime.now().isoformat()
            }
        )
        
        # If highly successful, also add to shared learning
        if success_score >= 0.9:
            self.store.put(
                ("agents", "shared_episodes"),
                episode_id,
                {
                    "source_agent": agent_name,
                    "query": query,
                    "response": response,
                    "success_score": success_score,
                    "what_worked": "Extracted from agent reflection"
                }
            )
```

**Consistency & Concurrency:**

1. **User Profile Updates**
   - Use row-level locking in PostgreSQL
   - Timestamp all updates, use "last write wins" strategy
   - Invalidate Redis cache on write

2. **Episodic Memory Collection**
   - Each agent writes independently (no conflicts)
   - Background job promotes high-quality episodes to shared namespace

3. **Procedural Memory Updates**
   - Version all instruction changes (v1, v2, v3...)
   - Atomic updates with compare-and-swap
   - Gradual rollout (A/B test new instructions)

**Memory Cleanup Policies:**

```python
# Automated cleanup job (runs daily)
def cleanup_memory_store(store: PostgresStore):
    now = datetime.now()
    
    # 1. Delete inactive conversation threads (30 days)
    for thread in store.search(("*", "*", "history")):
        if (now - datetime.fromisoformat(thread.value["last_updated"])).days > 30:
            store.delete(thread.namespace, thread.key)
    
    # 2. Archive old episodes (2 years)
    for episode in store.search(("*", "episodes")):
        age_days = (now - datetime.fromisoformat(episode.value["timestamp"])).days
        if age_days > 730:
            # Move to archive storage (S3) and delete from hot store
            archive_episode(episode)
            store.delete(episode.namespace, episode.key)
    
    # 3. Prune low-value episodes (quarterly)
    for episode in store.search(("*", "episodes")):
        if episode.value.get("success_score", 0) < 0.5:
            age_days = (now - datetime.fromisoformat(episode.value["timestamp"])).days
            if age_days > 90:  # Keep low-value for 90 days only
                store.delete(episode.namespace, episode.key)
    
    # 4. User profiles: NEVER auto-delete (compliance + UX)
    # 5. Procedural instructions: Keep all versions (audit trail)
```

**Deployment Architecture:**

```
                    ┌──────────────────┐
                    │   Load Balancer  │
                    └────────┬─────────┘
                             │
            ┌────────────────┼────────────────┐
            │                │                │
    ┌───────▼──────┐ ┌──────▼───────┐ ┌─────▼────────┐
    │   Agent      │ │   Agent      │ │   Agent      │
    │ Instance 1   │ │ Instance 2   │ │ Instance 3   │
    └───────┬──────┘ └──────┬───────┘ └─────┬────────┘
            │                │                │
            └────────────────┼────────────────┘
                             │
                    ┌────────▼─────────┐
                    │  Redis Cache     │
                    │  (Hot data)      │
                    └────────┬─────────┘
                             │
                    ┌────────▼─────────┐
                    │  PostgreSQL      │
                    │  + pgvector      │
                    │  (Persistent)    │
                    └──────────────────┘
```

**Key Principles:**

1. **User data = PostgreSQL** (persistent, multi-instance)
2. **Shared knowledge = PostgreSQL** (semantic search with pgvector)
3. **Hot cache = Redis** (performance)
4. **Cross-agent learning = Shared namespaces**
5. **Compliance = Never auto-delete user profiles or audit trails**

---
## 🏗️ Activity #2: Investment Memory Dashboard

Build an investment tracking system that:
1. Tracks investment metrics over time (portfolio value, risk score, allocation drift)
2. Uses semantic memory to find relevant investment advice
3. Uses episodic memory to recall what advisory approaches worked before
4. Uses procedural memory to adapt advice style
5. Provides a synthesized "investment summary"

### Requirements:
- Store at least 3 investment metrics per user
- Track metrics over multiple "days" (simulated)
- Agent should reference historical data in responses
- Generate a personalized investment summary

In [ ]:
from datetime import datetime, timedelta
import random

def log_investment_metric(store, user_id: str, date: str, metric_type: str, value: float, notes: str = ""):
    """Log an investment metric for a user."""
    namespace = (user_id, "metrics")
    metric_id = f"{metric_type}_{date}"
    
    store.put(
        namespace,
        metric_id,
        {
            "date": date,
            "metric_type": metric_type,
            "value": value,
            "notes": notes,
            "timestamp": datetime.now().isoformat()
        }
    )
    print(f"Logged {metric_type}={value} for {user_id} on {date}")


def get_investment_history(store, user_id: str, metric_type: str = None, days: int = 7) -> list:
    """Get investment history for a user."""
    namespace = (user_id, "metrics")
    
    # Get all metrics for user
    all_metrics = list(store.search(namespace))
    
    # Filter by metric type if specified
    if metric_type:
        all_metrics = [m for m in all_metrics if m.value["metric_type"] == metric_type]
    
    # Filter by date range
    cutoff_date = (datetime.now() - timedelta(days=days)).date().isoformat()
    recent_metrics = [m for m in all_metrics if m.value["date"] >= cutoff_date]
    
    # Sort by date
    recent_metrics.sort(key=lambda x: x.value["date"])
    
    return recent_metrics


store_activity2 = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
    }
)

# Simulate user profile
user_id_dash = "user_dashboard_test"
store.put(
    (user_id_dash, "profile"),
    "name",
    {"value": "Emma Thompson"}
)
store.put(
    (user_id_dash, "profile"),
    "risk_tolerance",
    {"value": "moderate"}
)
store.put(
    (user_id_dash, "profile"),
    "portfolio_size",
    {"value": "$750K"}
)

# Simulate a week of investment metrics
base_portfolio_value = 750000
base_risk_score = 6.5  # out of 10
base_allocation_drift = 2.0  # percentage points from target

for i in range(7):
    date = (datetime.now() - timedelta(days=6-i)).date().isoformat()
    
    # Simulate market volatility
    daily_return = random.uniform(-0.02, 0.03)  # -2% to +3%
    portfolio_value = base_portfolio_value * (1 + daily_return * i/7)
    
    # Risk score increases with market volatility
    risk_score = base_risk_score + random.uniform(-0.5, 0.5)
    
    # Allocation drift increases over time without rebalancing
    allocation_drift = base_allocation_drift + (i * 0.3)
    
    log_investment_metric(store_activity2, user_id_dash, date, "portfolio_value", portfolio_value, 
                         f"Market {'up' if daily_return > 0 else 'down'}")
    log_investment_metric(store_activity2, user_id_dash, date, "risk_score", risk_score,
                         "Risk assessment based on VaR and portfolio beta")
    log_investment_metric(store_activity2, user_id_dash, date, "allocation_drift", allocation_drift,
                         "Drift from 60/40 target allocation")

print("\n" + "="*60)
print("Created 7 days of investment metrics for Emma Thompson")
print("="*60)


# Add some investment knowledge to semantic memory
investment_knowledge_items = [
    ("advice_rebalance", {
        "text": "When portfolio allocation drifts more than 5% from target, consider rebalancing to maintain risk profile and ensure alignment with investment goals. Rebalancing helps buy low and sell high systematically."
    }),
    ("advice_volatility", {
        "text": "During periods of high volatility (VIX > 25), consider adding defensive positions or tail risk hedges. Quality bonds and gold often provide portfolio stability in turbulent markets."
    }),
    ("advice_risk_score", {
        "text": "A risk score above 7 suggests elevated portfolio volatility. For moderate-risk investors, consider reducing equity exposure or adding low-correlation alternatives like reinsurance or managed futures."
    })
]

advice_namespace = ("investment", "advisory_knowledge")
for key, value in investment_knowledge_items:
    store_activity2.put(advice_namespace, key, value)

# Add episodic memory - successful past advisory approaches
episode_rebalancing = {
    "text": "User had 8% allocation drift and received rebalancing advice",
    "situation": "Portfolio drift exceeding comfort zone",
    "advice_given": "Created phased rebalancing plan over 3 months to minimize tax impact and market timing risk",
    "user_response": "Implemented successfully, reported reduced anxiety about portfolio"
}

store_activity2.put(
    ("agent", "episodes"),
    "episode_rebalancing_success",
    episode_rebalancing
)

print("Added investment knowledge and episodic memories to store")


# Build an investment dashboard agent that:
#   - Retrieves user's investment history
#   - Searches for relevant advice based on patterns
#   - Uses episodic memory for what worked before
#   - Generates a personalized summary

class DashboardState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def investment_dashboard_agent(state: DashboardState, config: RunnableConfig, *, store: BaseStore):
    """Investment dashboard agent with all 5 memory types."""
    user_id = state["user_id"]
    user_query = state["messages"][-1].content
    
    # 1. LONG-TERM: Get user profile
    profile_items = list(store.search((user_id, "profile")))
    profile = {p.key: p.value["value"] for p in profile_items}
    
    # 2. HISTORICAL DATA: Get investment metrics (last 7 days)
    metrics = get_investment_history(store, user_id, days=7)
    
    # Organize metrics by type
    portfolio_values = [m for m in metrics if m.value["metric_type"] == "portfolio_value"]
    risk_scores = [m for m in metrics if m.value["metric_type"] == "risk_score"]
    allocation_drifts = [m for m in metrics if m.value["metric_type"] == "allocation_drift"]
    
    # Calculate trends
    if len(portfolio_values) >= 2:
        value_change = portfolio_values[-1].value["value"] - portfolio_values[0].value["value"]
        value_pct_change = (value_change / portfolio_values[0].value["value"]) * 100
    else:
        value_pct_change = 0
    
    current_risk = risk_scores[-1].value["value"] if risk_scores else 0
    current_drift = allocation_drifts[-1].value["value"] if allocation_drifts else 0
    
    # Build metrics summary
    metrics_summary = f"""
PORTFOLIO METRICS (Last 7 Days):
- Portfolio Value: ${portfolio_values[-1].value['value']:,.0f} ({value_pct_change:+.2f}%)
- Current Risk Score: {current_risk:.1f}/10
- Allocation Drift: {current_drift:.1f}% from target

WEEKLY TREND:
- {'Portfolio grew' if value_pct_change > 0 else 'Portfolio declined'} by {abs(value_pct_change):.2f}%
- Risk score: {'increased' if current_risk > 6.5 else 'stable or decreased'}
- Allocation drift: {'increasing - rebalancing may be needed' if current_drift > 5 else 'within acceptable range'}
"""
    
    # 3. SEMANTIC: Search for relevant advice based on current metrics
    advice_queries = []
    if current_drift > 5:
        advice_queries.append("portfolio allocation drift rebalancing")
    if current_risk > 7:
        advice_queries.append("high risk score portfolio volatility")
    if value_pct_change < -3:
        advice_queries.append("portfolio decline market downturn")
    
    relevant_advice = []
    for query in advice_queries:
        results = store.search(("investment", "advisory_knowledge"), query=query, limit=1)
        relevant_advice.extend(results)
    
    advice_text = "\n\n".join([f"- {r.value['text']}" for r in relevant_advice]) if relevant_advice else "No specific advice triggered."
    
    # 4. EPISODIC: Search for similar past situations
    situation_query = f"portfolio drift {current_drift}% risk score {current_risk}"
    similar_episodes = store.search(("agent", "episodes"), query=situation_query, limit=1)
    
    episodic_context = ""
    if similar_episodes:
        ep = similar_episodes[0].value
        episodic_context = f"""
LEARNING FROM PAST SUCCESS:
Previous similar situation: {ep['situation']}
What worked: {ep['advice_given']}
User outcome: {ep['user_response']}
"""
    
    # 5. PROCEDURAL: Build system prompt with all context
    system_prompt = f"""You are an Investment Dashboard Assistant for {profile.get('name', 'the user')}.

USER PROFILE:
{chr(10).join([f"- {k}: {v}" for k, v in profile.items()])}

{metrics_summary}

RELEVANT INVESTMENT ADVICE:
{advice_text}

{episodic_context}

Your task: Provide a concise, actionable investment summary. Reference specific metrics and trends. 
Suggest 1-2 concrete next steps based on the data. Be supportive but data-driven."""
    
    # Generate response
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the dashboard graph
builder_dashboard = StateGraph(DashboardState)
builder_dashboard.add_node("dashboard", investment_dashboard_agent)
builder_dashboard.add_edge(START, "dashboard")
builder_dashboard.add_edge("dashboard", END)

dashboard_graph = builder_dashboard.compile(
    checkpointer=MemorySaver(),
    store=store_activity2
)

print("\n" + "="*60)
print("Investment Dashboard Agent ready!")
print("="*60)


# Step 4: Test the dashboard
print("\n" + "="*60)
print("TEST 1: Portfolio Performance Summary")
print("="*60)

response1 = dashboard_graph.invoke(
    {
        "messages": [HumanMessage(content="Give me a summary of my portfolio performance this week.")],
        "user_id": user_id_dash
    },
    {"configurable": {"thread_id": "dashboard_thread_1"}}
)

print(f"\nUser: Give me a summary of my portfolio performance this week.")
print(f"\nDashboard Agent:\n{response1['messages'][-1].content}")

print("\n" + "="*60)
print("TEST 2: Addressing Portfolio Volatility")
print("="*60)

response2 = dashboard_graph.invoke(
    {
        "messages": [HumanMessage(content="My portfolio has been volatile lately and I'm seeing allocation drift. What should I do?")],
        "user_id": user_id_dash
    },
    {"configurable": {"thread_id": "dashboard_thread_2"}}
)

print(f"\nUser: My portfolio has been volatile lately and I'm seeing allocation drift. What should I do?")
print(f"\nDashboard Agent:\n{response2['messages'][-1].content}")

print("\n" + "="*60)
print("DASHBOARD SUMMARY:")
print("The agent successfully used:")
print("✓ Long-term memory: User profile (name, risk tolerance)")
print("✓ Historical metrics: 7 days of portfolio/risk/drift data")
print("✓ Semantic memory: Retrieved relevant investment advice")
print("✓ Episodic memory: Referenced past successful rebalancing advice")
print("✓ Short-term memory: Maintains conversation context across questions")
print("="*60)

---
## Summary

In this module, we explored the **5 memory types** from the CoALA framework:

| Memory Type | LangGraph Component | Scope | Investment Use Case |
|-------------|---------------------|-------|-------------------|
| **Short-term** | `MemorySaver` + `thread_id` | Within thread | Current consultation |
| **Long-term** | `InMemoryStore` + namespaces | Across threads | User profile, goals, constraints |
| **Semantic** | Store + embeddings + `search()` | Across threads | Investment knowledge retrieval |
| **Episodic** | Store + few-shot examples | Across threads | Past successful interactions |
| **Procedural** | Store + self-reflection | Across threads | Self-improving instructions |

### Key Takeaways:

1. **Memory transforms chatbots into advisors** - Persistence enables personalization
2. **Different memory types serve different purposes** - Choose based on your use case
3. **Context management is critical** - Trim and summarize to stay within limits
4. **Episodic memory enables learning** - Show, don't just tell
5. **Procedural memory enables adaptation** - Agents can improve themselves

### Production Considerations:

- Use `PostgresSaver` instead of `MemorySaver` for persistent checkpoints
- Use `PostgresStore` instead of `InMemoryStore` for persistent long-term memory
- Consider TTL (Time-to-Live) policies for automatic memory cleanup
- Implement proper access controls for user data
- Ensure compliance with financial regulations for investment advisory data

### Further Reading:

- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)
- [CoALA Paper](https://arxiv.org/abs/2309.02427) - Cognitive Architectures for Language Agents
- [LangGraph Platform](https://docs.langchain.com/langgraph-platform/) - Managed infrastructure for production